# Amazon SageMaker Model Bias Monitors: Fairness and Explainability with SageMaker Clarify

Amazon SageMaker Clarify membantu meningkatkan model <i>machine learning</i> Anda dengan mendeteksi potensi bias dan membantu menjelaskan prediksi yang dibuat model. Fungsionalitas kelayakan dan kemampuan menjelaskan yang disediakan oleh SageMaker Clarify mengambil langkah untuk memungkinkan pelanggan AWS membangun model <i>machine learning</i> yang tepercaya dan dapat dipahami. Produk ini dilengkapi dengan alat untuk membantu Anda dengan tugas-tugas berikut.

- Mengukur bias yang dapat terjadi selama setiap tahap siklus hidup ML (pengumpulan data, pelatihan dan penyetelan model, serta <i>monitoring</i> model ML yang di-<i>deploy</i> untuk inferensi).
- Menghasilkan laporan tata kelola model yang menargetkan tim risiko dan kepatuhan serta regulator eksternal.
- Memberi penjelasan tentang data, model, dan pemantauan yang digunakan untuk menilai prediksi.


## Impor

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

### Konfigurasi untuk tugas ini

In [ ]:
# Replace below <bucket_name> with the value the modelArtifactBucket value at the left of the instruction page
bucket = '<bucket_name>'

# Get production endpoint name
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Get production model name
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']


print(f"Bucket Name :{bucket}")
print(f"Production Endpoint Name:{endpoint_name}")
print(f"Production Model Name:{model_name}")

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

In [ ]:
print(f"Demo Bucket: {bucket}")
prefix = f'sagemaker/DEMO-ClarifyModelMonitor-{datetime.now():%Y%m%d%H%M%S}'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.xlarge"

### <i>File</i> data

Baca <i>file</i> data

In [ ]:
test_dataset = "data/test.csv"
validation_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

### Mengaktifkan Penangkapan Data di <i>endpoint</i>

Untuk mengaktifkan penangkapan data untuk tugas <i>monitoring</i>, tentukan opsi pengambilan baru di sini yang disebut DataCaptureConfig, gunanya untuk mengaktifkan penangkapan <i>payload</i> permintaan dan <i>payload </i>respons<i> endpoint.</i> Konfigurasi penangkapan berlaku untuk semua varian. 

Operasi ini biasanya memakan waktu 10-15 menit.

**Catatan:** Operasi ini dapat diotomatisasi melalui cloudformation (peninjau - harap sarankan)

In [ ]:
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])



# Now it is time to apply the new configuration and wait for it to be applied
predictor = Predictor(endpoint_name=endpoint_name)
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sagemaker_session.wait_for_endpoint(endpoint=endpoint_name)

### Menguji <i>Endpoint</i> dengan lalu lintas sampel

In [ ]:
# Here we are randomly picking subset of data from test datasets.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

In [ ]:
# Invoke Endpoint with test data for a quick test
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## Melihat data yang ditangkap

Sekarang cantumkan <i>file</i> penangkapan data yang disimpan di Amazon S3. Anda akan melihat <i>file</i> yang berbeda dari periode waktu berbeda yang diatur berdasarkan jam saat pemanggilan terjadi. Format jalur Amazon S3 adalah:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
print(result)
str_error = "waiting for the file to load..."
while str_error:
    try:
        capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
        print(capture_files)
        str_error=None
    except:
        print(str_error)
        time.sleep(10)
        contine
print("Found Capture Files:")
print("\n ".join(capture_files))

Selanjutnya, lihat konten dari <i>file</i> penangkapan tunggal. Di sini, Anda akan melihat semua data yang ditangkap dalam<i> file</i> berformat baris JSON khusus Amazon SageMaker. Lihat sekilas beberapa baris pertama dalam<i> file</i> yang ditangkap.

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

Terakhir, konten satu baris disajikan di bawah ini dalam <i>file </i>berformat JSON sehingga Anda dapat mengamati sedikit lebih baik.

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Penyiapan Amazon SageMaker Clarify

**Catatan:** Kode sampel di bawah ini diambil dari [kode sampel](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb) yang disediakan Amazon sagemaker. Beberapa kode diperbarui untuk persyaratan lab.

In [ ]:
# Run SageMaker Clarify processor using the following command
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

Unggah set data pengujian dan pelatihan ke s3. _train_uri_ dan _test_uri_ akan digunakan untuk menarik data secara langsung dari konfigurasi data penulisan S3 dan konfigurasi model di langkah selanjutnya.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

### Mendeteksi Bias

SageMaker Clarify membantu Anda mendeteksi kemungkinan bias pra- dan pasca-pelatihan menggunakan berbagai metrik.

### Menulis DataConfig dan ModelConfig

Objek **DataConfig** mengomunikasikan beberapa informasi dasar mengenai I/O data ke SageMaker Clarify. Anda akan menentukan tempat menemukan set data input, tempat menyimpan <i>output</i>, kolom target (label), nama <i>header</i>, dan jenis set data.

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

Objek **ModelConfig** mengomunikasikan informasi mengenai model terlatih Anda. Untuk menghindari lalu lintas tambahan ke model produksi Anda, SageMaker Clarify menyiapkan dan meruntuhkan <i>endpoint</i> khusus saat pemrosesan.

- **instance_type** dan **instance_count** menentukan tipe instans pilihan Anda dan jumlah instans yang digunakan untuk menjalankan model Anda selama pemrosesan SageMaker Clarify. Set data pengujian berukuran kecil sehingga satu instans standar sudah cukup untuk menjalankan instans ini. Jika Anda memiliki set data kompleks yang besar, Anda mungkin ingin menggunakan tipe instans yang lebih baik untuk mempercepat, atau menambahkan lebih banyak instans untuk mengaktifkan paralelisasi Spark.

- **accept_type** menunjukkan format <i>payload</i> respons <i>endpoint</i>, dan **content_type** menunjukkan format <i>payload</i> permintaan ke <i>endpoint</i>.

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

**ModelPredictedLabelConfig** memberikan informasi mengenai format prediksi Anda. Model <i>Random Forest </i>mengeluarkan probabilitas sampel, jadi SageMaker Clarify memanggil <i>endpoint</i> kemudian menggunakan probability_threshold untuk mengonversi probabilitas ke label biner untuk analisis bias. Prediksi di atas ambang batas diartikan sebagai nilai label 1 dan di bawah atau sama dengan sebagai nilai label 0.

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### Menulis BiasConfig
SageMaker Clarify juga membutuhkan informasi mengenai kolom sensitif (faset), fitur sensitif (facet_values_or_threshold, dan hasil yang diinginkan (label_values_or_threshold). SageMaker Clarify dapat menangani data kategorikal dan berkelanjutan untuk facet_values_or_threshold dan label_values_or_threshold. Dalam hal ini kami menggunakan data mutlak.

Kami menentukan informasinya di API BiasConfig. Di sini hasil positifnya adalah Churn = 0, _Account Length_ merupakan kategori sensitif, dan responden _Day Calls_ merupakan grup sensitif, group_name digunakan untuk membentuk subgrup untuk pengukuran.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

**Bias Pra-pelatihan**

Bias dapat hadir dalam data Anda sebelum pelatihan model apa pun terjadi. Memeriksa data Anda untuk bias sebelum pelatihan dimulai dapat membantu mendeteksi kesenjangan pengumpulan data, memberi tahu rekayasa fitur Anda, dan membantu memahami bias sosial yang mungkin dicerminkan oleh data.

Mengomputasi metrik bias pra-pelatihan tidak memerlukan model terlatih.

**Bias Pasca-pelatihan**

Mengomputasi metrik bias pasca-pelatihan memerlukan model terlatih.

Data pelatihan yang tidak bias (yang ditentukan oleh konsep kelayakan yang diukur dengan metrik bias) masih dapat menghasilkan prediksi model yang bias setelah pelatihan. Baik terjadinya bias tergantung pada beberapa faktor termasuk pilihan hiperparameter atau tidak.

Anda dapat menjalankan opsi ini secara terpisah menggunakan run_pre_training_bias() dan run_post_training_bias(), atau bersamaan dengan run_bias() seperti yang ditunjukkan di bawah ini.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

**Melihat Laporan Bias**

Di Studio, Anda dapat melihat hasilnya di bawah <i>tab</i> eksperimen.

<img src="./recordings/bias_report.gif">

Setiap metrik bias memiliki penjelasan mendetail dengan contoh yang dapat Anda jelajahi.

<img src="./recordings/bias_detail.gif">

Anda juga dapat meringkas hasilnya dalam tabel praktis!

<img src="./recordings/bias_report_chart.gif">



Jika Anda belum menjadi pengguna Studio, Anda dapat mengakses laporan bias dalam format pdf, html, dan ipynb di <i>bucket</i> S3 berikut:

In [ ]:
bias_report_output_path

### Menjelaskan Prediksi

Ada kebutuhan bisnis yang berkembang dan peraturan perundang-undangan yang memerlukan penjelasan penyebab model membuat keputusan tersebut. SageMaker Clarify menggunakan SHAP untuk menjelaskan kontribusi yang dibuat oleh setiap fitur input terhadap keputusan akhir.

Algoritma SHAP Kernel membutuhkan <i>baseline</i> (juga dikenal sebagai set data latar belakang). Jenis set data dasar harus sama dengan dataset_type dari DataConfig, dan sampel <i>baseline</i> hanya boleh menyertakan fitur. Menurut definisi, <i>baseline</i> harus berupa URI S3 ke <i>file</i> set data <i>baseline</i>, atau daftar sampel di tempat. Dalam hal ini kami memilih yang terakhir, dan menempatkan sampel pertama dari set data uji ke dalam daftar.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

#### Melihat Laporan Kemampuan Menjelaskan
Seperti laporan bias, Anda dapat melihat laporan kemampuan menjelaskan di Studio di bawah<i> tab</i> eksperimen


<img src="./recordings/explainability_detail.gif">

<i>Tab</i> Wawasan Model berisi tautan langsung ke laporan dan wawasan model.

Jika Anda belum menjadi pengguna Studio, seperti halnya Laporan Bias, Anda dapat mengakses laporan ini di <i>bucket</i> S3 berikut.

In [ ]:
explainability_output_path

### Pembersihan

Terakhir, jangan lupa untuk membersihkan sumber daya yang kami siapkan dan gunakan untuk demo ini!

In [ ]:
session.delete_model(model_name)